In [ ]:
# 1
import os
import json
from requests import post, exceptions
from dotenv import load_dotenv

load_dotenv()


api_key = os.getenv("ragie_api")
file_path = "cagatay_kilinc_resume.pdf"

try:
    with open(file_path, "rb") as f:
        content = f.read()
except FileNotFoundError:
    print(f"Dosya bulunamadı: {file_path}")
    exit()

metadata = {"title": "resume", "environment": "material_costs"}

data = {
    "file": (file_path, content),
    "metadata": json.dumps(metadata)
}

headers = {
    "Authorization": f"Bearer {api_key}",
    "Accept": "application/json"
}

try:
    response = post("https://api.ragie.ai/documents", data=data, headers=headers)
    response.raise_for_status()  
    print("Dosya başarıyla yüklendi!")
except exceptions.RequestException as e:
    print(f"Yükleme başarısız: {e}")


promblems:
                                    
1- Ragie can automatically delete our  API key. We have to renew the API key almost every 5 minutes.

2- We can't see the uploaded file on a dashboard. we cannot choice chunked doc to rag. 

3- When trying to upload a second file, we're getting an authentication error.

4- It's returning error codes that are different from those documented.

Here's what I figured out about Ragie:

1-  Ragie  makes a document ready for rag. 

3- No llm.

In [ ]:
#2

import json
import requests



data = {
    "query": "davos",
    "filter": {
        "environment": "tutorial"
    }
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

try:
  response = requests.post("https://api.ragie.ai/retrievals", json=data, headers=headers)
  response.raise_for_status()  
  data = response.json()
  print(data)
except requests.exceptions.RequestException as e:
  print(f"Error: {e}")


In [ ]:
#3

import requests
import json



query = ""


headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
data = {
    "query": query,
    "filters": {
        "environment": "tutorial"
    }
}

try:
  response = requests.post("https://api.ragie.ai/retrievals", json=data, headers=headers)
  response.raise_for_status() 
  ragie_data = response.json()
except requests.exceptions.RequestException as e:
  print(f"Error: Failed to retrieve data from Ragie API: {e}")
  exit(1)



from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


chunk_text = [chunk["text"] for chunk in ragie_data["scored_chunks"]]

chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key="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" 
)

system = "sana verilen {chunk_text} bu bilgilere göre soruları net bir şekilde cevapla"
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])
chain = prompt | chat


In [ ]:


chain.invoke({"text": query,"chunk_text": chunk_text})

In [35]:
import requests

while True:
  headers = {"Authorization": f"Bearer {api_key}"}
  response = requests.get("https://api.ragie.ai/documents", headers=headers)
  response.raise_for_status() 

  payload = response.json()

  for document in payload["documents"]:
    if document["metadata"]["environment"] != "tutorial":
      continue

    response = requests.delete(
        f"https://api.ragie.ai/documents/{document['id']}", headers=headers
    )
    response.raise_for_status()  
    print(f"Deleted document {document['id']}")

  if not payload["pagination"]["next_cursor"]:
    print("No more documents\n")
    break

print("Finished deleting documents.")


HTTPError: 401 Client Error: Unauthorized for url: https://api.ragie.ai/documents